<a href="https://colab.research.google.com/github/botmaker00/Rex-Deploy/blob/main/rex_hk_deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>REX BOTS - ENCODER BOT HEROKU DEPLOY (With UPSTREAM Support)</center></h1>
<center><img src="https://files.catbox.moe/4w80av.jpg" height="220"></center>

**Channel:** @RexBots_Official | **Support:** @rexbotschat  
**Updated:** Added UPSTREAM_REPO & UPSTREAM_BRANCH support

---

In [ ]:
#@title 1. Heroku Login
Heroku_Email = "" #@param {type:"string"}
Heroku_API = "" #@param {type:"string"}

!curl -s https://cli-assets.heroku.com/install.sh | sh
from IPython.display import clear_output, HTML
clear_output()
print("Heroku CLI Installed")

if not Heroku_Email or not Heroku_API:
    raise Exception("Fill Heroku Email & API Key!")

import os
netrc = f"machine api.heroku.com\n  login {Heroku_Email}\n  password {Heroku_API}\nmachine git.heroku.com\n  login {Heroku_Email}\n  password {Heroku_API}"
with open(os.path.expanduser("~/.netrc"), "w") as f:
    f.write(netrc)
os.chmod(os.path.expanduser("~/.netrc"), 0o600)

!git config --global user.email {Heroku_Email}
!git config --global user.name "RexBot"

display(HTML("<h3>Heroku Login Successful!</h3>"))

In [ ]:
#@title 2. Rex Bots Config Setup (Now with UPSTREAM_REPO & UPSTREAM_BRANCH)
App_Name = "rexbot1" #@param {type:"string"}

#@markdown ### Custom Upstream Repo (Leave empty to use default)
UPSTREAM_REPO = "" #@param {type:"string"}
#@markdown Default: https://github.com/SilentDemonSD/Rex-Bots

#@markdown ### Custom Branch (Leave empty for main/master)
UPSTREAM_BRANCH = "" #@param {type:"string"}
#@markdown Common: main, master, v2, beta etc.

#@markdown ---
#@markdown ### Config Methods → Choose Any One

#@markdown ### Method 1: Fill Manually
API_ID = 12345678 #@param {type:"integer"}
API_HASH = "" #@param {type:"string"}
BOT_TOKEN = "" #@param {type:"string"}
OWNER_ID = 1234567890 #@param {type:"integer"}
SUDO_USERS = "" #@param {type:"string"}
EVERYONE_CHATS = "" #@param {type:"string"}
LOG_CHANNEL = -1001234567890 #@param {type:"integer"}
SESSION_NAME = "encoderbot" #@param {type:"string"}
MONGO_URI = "" #@param {type:"string"}
DOWNLOAD_DIR = "VideoEncoder/downloads/" #@param {type:"string"}
ENCODE_DIR = "VideoEncoder/encodes/" #@param {type:"string"}
INDEX_URL = "" #@param {type:"string"}
DRIVE_DIR = "" #@param {type:"string"}

#@markdown ### Method 2: Gist Raw URL
Gist_URL = "" #@param {type:"string"}

#@markdown ### Method 3: Upload config.env
Upload_Config = False #@param {type:"boolean"}

import os, shutil
if os.path.exists(App_Name):
    shutil.rmtree(App_Name)

# Use custom repo/branch if provided, else default
repo_url = UPSTREAM_REPO.strip() or "https://github.com/SilentDemonSD/Rex-Bots"
branch = UPSTREAM_BRANCH.strip() or "main"

!git clone --branch {branch} {repo_url} {App_Name}
%cd {App_Name}

# Clean junk
!rm -f README.md *.ipynb *.jpg 2>/dev/null

if Upload_Config:
    from google.colab import files
    uploaded = files.upload()
    for fn in uploaded.keys():
        os.rename(fn, "config.env")
    print("config.env uploaded")

elif Gist_URL.strip():
    !curl -L -o config.env "{Gist_URL}"
    print("config.env downloaded from Gist")

else:
    config_text = f'''# Basics
API_ID = {API_ID}
API_HASH = "{API_HASH}"
BOT_TOKEN = "{BOT_TOKEN}"

# Authorization
OWNER_ID = {OWNER_ID}
SUDO_USERS = {SUDO_USERS}
EVERYONE_CHATS = {EVERYONE_CHATS}

# Log
LOG_CHANNEL = {LOG_CHANNEL}

# Database
SESSION_NAME = "{SESSION_NAME}"
MONGO_URI = "{MONGO_URI}"

# Paths
DOWNLOAD_DIR = "{DOWNLOAD_DIR}"
ENCODE_DIR = "{ENCODE_DIR}"
INDEX_URL = "{INDEX_URL}"
DRIVE_DIR = "{DRIVE_DIR}"
'''
    with open("config.env", "w") as f:
        f.write(config_text)
    print("config.env created manually")

# Add UPSTREAM variables if user filled them
if UPSTREAM_REPO.strip():
    with open("config.env", "a") as f:
        f.write(f"\nUPSTREAM_REPO = {UPSTREAM_REPO}\n")
        if UPSTREAM_BRANCH.strip():
            f.write(f"UPSTREAM_BRANCH = {UPSTREAM_BRANCH}\n")
        else:
            f.write(f"UPSTREAM_BRANCH = main\n")

%cd ..
print("\nConfigured Apps:")
!ls -d */

In [ ]:
#@title 3. Create Heroku Apps (Optional)
App_Names = "" #@param {type:"string"}
Region = "eu" #@param ["eu", "us"]
Team = "" #@param {type:"string"}

team_flag = f"--team {Team}" if Team else ""
for name in App_Names.split():
    name = name.strip()
    if name:
        !heroku create --region {Region} --stack container {team_flag} {name}
        print(f"Created → {name}")

In [ ]:
#@title 4. Deploy All Apps
Deploy_List = "" #@param {type:"string"}

for app in Deploy_List.split():
    app = app.strip()
    if os.path.isdir(app):
        %cd {app}
        !git add . -f
        !git commit -m "Rex Bots Deploy - $(date)" --allow-empty
        !heroku git:remote -a {app}
        !git push heroku main -f
        print(f"{app} → Deployed!")
        %cd ..
    else:
        print(f"Folder {app} not found!")

In [ ]:
#@title 5. View Logs
App = "" #@param {type:"string"}
!heroku logs -t -a {App}

In [ ]:
#@title 6. Logout
!heroku logout